# **Programming Assessment \#4**

Names: Go, Wilfred | Sibug, Jordan | Sy, James Matthew

More information on the assessment is found in our Canvas course. Link: https://dlsu.instructure.com/courses/93383/assignments/739602

# **Load Data**

*While you don't have to separate your code into blocks, it might be easier if you separated loading your data from actually implementation of your code. Consider placing all loading of data into the code block below.*

In [1]:
# load the words from the Gutenberg Document
import nltk
nltk.download('gutenberg')
nltk.corpus.gutenberg.fileids()

from collections import Counter

# print("Extracting all documents from NLTK's Project Gutenberg Collection...")
all_words = Counter()
for filename in nltk.corpus.gutenberg.fileids():
    words = [word.lower() for word in nltk.corpus.gutenberg.words(filename)]
    all_words.update(words)
#   print("%s; tokens: %d; vocab: %d" % (filename, len(words), len(set(words))))

# print("Overall Statistics")
# total_tokens = sum(all_words.values())
total_types = len(all_words)

# print("Total tokens: %d" % total_tokens)
# print("Total vocabulary / type: %d" % total_types)
# print("Type/token ratio: %.4f" % (total_types / total_tokens))
# print("Vocabulary richness: %.4f" % (total_types / (total_tokens ** (1/2))))

[nltk_data] Downloading package gutenberg to C:\Users\JAMES
[nltk_data]     SY\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
# print the word
print(words[500])
print(words[100])
print(words[664])
print(words[32])
print(words[10000])
print(words[5])

# get the count of the word
print(all_words[words[500]])

the
soul
destiny
i
near
walt
133583


# **Noisy Channel Model Implementation**

*Again, you don't have to follow this directly, but consider placing your implementation of the model in the code block below. And while we discussed the general approach in class, kindly describe how you decided to implement the spell correction model. Include any modifications your group made as well. This might be a good spot to place part of your write up.*

In [3]:
# taken from http://norvig.com/spell-correct.html
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    
    wordsAndEdits = {
        'del': deletes,
        'tra': transposes,
        'sub': replaces,
        'ins': inserts
    }
#     return set(deletes + transposes + replaces + inserts)
    return wordsAndEdits

In [10]:
def spellCorrect(word):
    # check if the word is in the corpus
    if word in words:
        return "No error"

    # word should be corrected
    # get possible words within edit distance 1
    candidates = edits1(word)

    candidatesProb = {
        'del': [],
        'tra': [],
        'sub': [],
        'ins': []
    }
#     print(wordsAndEdits)
    edit_types = ['del', 'tra', 'sub', 'ins']
    
# find the probability of the candidate in the corpus
    for etype in edit_types:
        for candidate in candidates[etype]:
            if candidate in words:
                candidatesProb[etype].append(all_words[candidate] / total_types)
            else:
                candidatesProb[etype].append(0)

# remove the candidates with 0 probability
    for etype in edit_types:
        i = 0    
        while i != len(candidates[etype]):
            if candidatesProb[etype][i] == 0:
                del candidates[etype][i]
                del candidatesProb[etype][i]
            else:
                i += 1
                
    print(candidates)
    print(candidatesProb)

In [11]:
word = input("Input: ")

# call functions, return answer
output = spellCorrect(word)
print("Output: " + str(output))

Input: hte
{'del': ['te', 'he', 'ht'], 'tra': ['the', 'het'], 'sub': ['ate', 'bte', 'cte', 'dte', 'ete', 'fte', 'gte', 'hte', 'ite', 'jte', 'kte', 'lte', 'mte', 'nte', 'ote', 'pte', 'qte', 'rte', 'ste', 'tte', 'ute', 'vte', 'wte', 'xte', 'yte', 'zte', 'hae', 'hbe', 'hce', 'hde', 'hee', 'hfe', 'hge', 'hhe', 'hie', 'hje', 'hke', 'hle', 'hme', 'hne', 'hoe', 'hpe', 'hqe', 'hre', 'hse', 'hte', 'hue', 'hve', 'hwe', 'hxe', 'hye', 'hze', 'hta', 'htb', 'htc', 'htd', 'hte', 'htf', 'htg', 'hth', 'hti', 'htj', 'htk', 'htl', 'htm', 'htn', 'hto', 'htp', 'htq', 'htr', 'hts', 'htt', 'htu', 'htv', 'htw', 'htx', 'hty', 'htz'], 'ins': ['ahte', 'bhte', 'chte', 'dhte', 'ehte', 'fhte', 'ghte', 'hhte', 'ihte', 'jhte', 'khte', 'lhte', 'mhte', 'nhte', 'ohte', 'phte', 'qhte', 'rhte', 'shte', 'thte', 'uhte', 'vhte', 'whte', 'xhte', 'yhte', 'zhte', 'hate', 'hbte', 'hcte', 'hdte', 'hete', 'hfte', 'hgte', 'hhte', 'hite', 'hjte', 'hkte', 'hlte', 'hmte', 'hnte', 'hote', 'hpte', 'hqte', 'hrte', 'hste', 'htte', 'hute',

# **Your Relfection / Takeaway / Analysis**

*Kindly place the rest of your write up. More information is found in the Canvas write up.*